In [2]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import sqlite3
import taxis
from utils import *
from visualize import *
from sklearn.linear_model import *
from sklearn.kernel_ridge import *
from sklearn.pipeline import *
from sklearn.preprocessing import *
from datetime import datetime
from sklearn.cluster import *
from sklearn.svm import *

DBNAME="../taxis.db"
LAT=1.380766
LON=103.891675
LAT1=1.303693
LON1=103.844952

In [3]:
holidays=['20170626']
def peak(h):
    return 1 if ((h>6 and h <9) or (h > 18)) else 0

def weekday(d):
    dt=datetime.strptime(d,"%Y%m%d")
    return 1 if datetime.isoweekday(dt)<6 else 0

df=taxis.get_df_all_taxi_data(DBNAME,LAT1,LON1)
df['Hour']=df.apply(lambda r:int(r['Time'][:2]),axis=1)
df['Minute']=df.apply(lambda r:int(r['Time'][2:]),axis=1)
df['PeakHour']=df.apply(lambda r: peak(r['Hour']),axis=1)
df['Weekday']=df.apply(lambda r: weekday(r['Date']),axis=1)
df['Holiday']=df.apply(lambda r: 1 if r['Date'] in holidays else 0,axis=1)


In [4]:
df.tail()

,Point,Date,Time,X,Y,TaxiCount,Hour,Minute,PeakHour,Weekday,Holiday
4119,4119,20170708,2330,20,5,20,23,30,1,0,0
4120,4120,20170708,2335,20,5,18,23,35,1,0,0
4121,4121,20170708,2340,20,5,25,23,40,1,0,0
4122,4122,20170708,2345,20,5,28,23,45,1,0,0
4123,4123,20170708,2350,20,5,23,23,50,1,0,0


In [5]:
h=df.quantile(q=0.7)['TaxiCount']
l=df.quantile(q=0.3)['TaxiCount']

In [6]:
df_sub=df.query('TaxiCount>@l and TaxiCount<@h')[['Point','Hour','Minute','PeakHour','Weekday','Holiday','TaxiCount']]
df_sub.head()

,Point,Hour,Minute,PeakHour,Weekday,Holiday,TaxiCount
0,0,9,0,0,0,0,58
2,2,9,10,0,0,0,64
30,30,11,30,0,0,0,63
32,32,11,40,0,0,0,62
33,33,11,45,0,0,0,62


In [7]:
df_sub.corr()

,Point,Hour,Minute,PeakHour,Weekday,Holiday,TaxiCount
Point,1.000000,0.061384,0.021504,0.090127,0.329195,-0.296673,-0.037342
Hour,0.061384,1.000000,-0.009911,0.405892,-0.031238,-0.078773,0.006524
Minute,0.021504,-0.009911,1.000000,0.020537,0.025925,0.009766,-0.045273
PeakHour,0.090127,0.405892,0.020537,1.000000,0.029964,-0.097658,0.010259
Weekday,0.329195,-0.031238,0.025925,0.029964,1.000000,0.147794,-0.043289
Holiday,-0.296673,-0.078773,0.009766,-0.097658,0.147794,1.000000,0.025157
TaxiCount,-0.037342,0.006524,-0.045273,0.010259,-0.043289,0.025157,1.000000


In [8]:
l=df['Point'].count()
c=int(l*0.8)
d=l-c
print c
print d
training=df_sub.head(c)
testing=df_sub.tail(d)

3299
825


In [ ]:
X=training.drop(['Point','TaxiCount'],axis=1).as_matrix()
y=np.array(training['TaxiCount'].tolist()).reshape(-1,1)

svc=SVC(kernel='poly').fit(X,y)
print svc

svc.score(X,y)
#plt.plot(y,color='g')
#plt.plot(svc.predict(X),color='r')
#plt.show()


/Users/hari/python/py2/taxidata/lib/python2.7/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
Xt=testing.drop(['Point','TaxiCount'],axis=1).as_matrix()
yt=np.array(testing['TaxiCount'].tolist()).reshape(-1,1)
P=svc.predict(Xt)
e=[x-y for x,y in zip(P,yt)]

#plt.plot(yt,color='g')
#plt.plot(P,color='b')
plt.plot(e,color='r')
plt.show()